# Random Forest Regression Example

## Boston housing prices
The objective is to predict the median price of a home in Boston.  The variables are crime rate, zoning information,
proportion of non-retail business, etc.  This dataset has median prices in Boston for 1972.  Even though the data is pretty old, the methodology for analytics is valid for more recent datasets.

<b>The purpose of this demonstration is to show the use of SAP HANA's Predictive Analytics Library to created Random forest model.</b>

The dataset is from Kaggle. https://www.kaggle.com/c/boston-housing. For tutorials use only.

## Housing Values in Suburbs of Boston in 1972

The <font color='red'>medv</font> variable is the target variable.
### Data description
The Boston data frame has 506 rows and 14 columns.
This data frame contains the following columns:
1. __crim__: per capita crime rate by town.
2. __zn__: proportion of residential land zoned for lots over 25,000 sq.ft.
3. __indus__: proportion of non-retail business acres per town.
4. __chas__: Charles River dummy variable (1 if tract bounds river; 0 otherwise).
5. __nox__: nitrogen oxides concentration (parts per 10 million).
6. __rm__: average number of rooms per dwelling.
7. __age__: proportion of owner-occupied units built prior to 1940.
8. __dis__: weighted mean of distances to five Boston employment centres.
9. __rad__: index of accessibility to radial highways.
10. __tax__: full-value property-tax rate per \$10000
11. __ptratio__: pupil-teacher ratio by town.
12. __black__: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town.
13. __lstat__: lower status of the population (percent).
14. __medv__: median value of owner-occupied homes in $1000s.
</td></tr></table>

### Factoids
The prices in Boston across years is below.  If we had a historical dataset, an analysis could be done to account for the macro trends as well.

The second graph shows the intuition we have with respect to prices in relation to crime rate.  It is expected that house prices will be lower in areas where crime rates are higher.

The third figure is a chart showing how inflation may affect prices.  So, for deeper analysis and prediction, we may want to consider inflation.

In this notebook, these factors are not considered.  They are here to demonstrate the need for deep domain analysis.

<table><tr>
<td><img src="images/boston_prices_by_year.png" alt="Boston home prices" title="Boston housing prices" style="float:left;" /></td>
<td><img src="images/Crime-Rate-and-Median-House-Prices.png" alt="Boston home prices" title="Boston housing prices"  /></td>
<td><img src="images/Inflation_Adjusted_Housing_Prices_1890_2006.jpg" alt="Inflation adjusted prices" title="Inflation adjusted prices" style="float:left;" />
</td></tr></table>


In this notebook, we will use the dataset for Boston housing prices and predict the price based on numerous factors.

In [1]:
from hana_ml import dataframe
from hana_ml.algorithms.pal import clustering
from hana_ml.algorithms.pal import trees
import numpy as np
import matplotlib.pyplot as plt
import logging

## Load data
The data is loaded into 4 tables, for full, training, validation, and test sets:
<li>BOSTON_HOUSING_PRICES</li>
<li>BOSTON_HOUSING_PRICES_TRAINING</li>
<li>BOSTON_HOUSING_PRICES_VALIDATION</li>
<li>BOSTON_HOUSING_PRICES_TEST</li>

To do that, a connection is created and passed to the loader.

There is a config file, config/e2edata.ini that controls the connection parameters and whether or not to reload the data from scratch.  In case the data is already loaded, there would be no need to load the data.  A sample section is below.  If the config parameter, reload_data is true then the tables for test, training, and validation are (re-)created and data inserted into them.

Although this ini file has other sections, please do not modify them. Only the [hana] section should be modified.

#########################<br>
[hana]<br>
url=host.sjc.sap.corp<br>
user=username<br>
passwd=userpassword<br>
port=3xx15<br>
#########################<br>

In [2]:
from data_load_utils import DataSets, Settings
url, port, user, pwd = Settings.load_config("../../config/e2edata.ini")
connection_context = dataframe.ConnectionContext(url, port, user, pwd)
full_tbl, training_tbl, validation_tbl, test_tbl = DataSets.load_boston_housing_data(connection_context)

# Create Data Frames
Create the data frames for the full, test, training, and validation sets.

Let us also do some dtaa exploration.

## Define Datasets - Training, validation, and test sets
Data frames are used keep references to data so computation on large data sets in HANA can happen in HANA.  Trying to bring the entire data set into the client will likely result in out of memory exceptions.

The original/full dataset is split into training, test and validation sets.  In the example below, they reside in different tables.

In [3]:
full_set = connection_context.table(full_tbl)
training_set = connection_context.table(training_tbl)
validation_set = connection_context.table(validation_tbl)
test_set = connection_context.table(test_tbl)

## Simple Exploration
Let us look at the number of rows in the data set

In [4]:
print('Number of rows in full set: {}'.format(full_set.count()))
print('Number of rows in training set: {}'.format(training_set.count()))
print('Number of rows in validation set: {}'.format(validation_set.count()))
print('Number of rows in test set: {}'.format(test_set.count()))

Number of rows in full set: 506
Number of rows in training set: 315
Number of rows in validation set: 64
Number of rows in test set: 127


### Let's look at the columns

In [5]:
print(full_set.columns)

['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'BLACK', 'LSTAT', 'MEDV', 'ID']


### Let's look at the data types

In [6]:
full_set.dtypes()

[('CRIM', 'DECIMAL', 12),
 ('ZN', 'DECIMAL', 7),
 ('INDUS', 'DECIMAL', 7),
 ('CHAS', 'SMALLINT', 5),
 ('NOX', 'DECIMAL', 10),
 ('RM', 'DECIMAL', 8),
 ('AGE', 'DECIMAL', 7),
 ('DIS', 'DECIMAL', 11),
 ('RAD', 'TINYINT', 3),
 ('TAX', 'SMALLINT', 5),
 ('PTRATIO', 'DECIMAL', 6),
 ('BLACK', 'DECIMAL', 9),
 ('LSTAT', 'DECIMAL', 7),
 ('MEDV', 'DECIMAL', 6),
 ('ID', 'INT', 10)]

### Set up the features and labels for the model

In [7]:
features=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'BLACK', 'LSTAT']
label='MEDV'

# Create model using training data
For demonstration, we will create two models, model and model_with_id, one where we have a unique id in the training set and one where there is none.

We are using Random Forest regression and SVM routines in this example

Documentation is <a href="https://help.sap.com/http.svc/rc/DRAFT/3f0dbe754b194c42a6bf3405697b711f/2.0.031/en-US/html/index.html">here</a>

## Preprocessing
SAP HANA Predictive Analytics Library takes DOUBLE and INTEGER data types for most numeric types.  Since we have DECIMALs and TINYINTs in our data set, we cast them to the types required by PAL.

In [8]:
# Cast to correct types so PAL can consume it.
dfts = training_set.cast(['CRIM', "ZN", "INDUS", "NOX", "RM", "AGE", "DIS", "PTRATIO", "BLACK", "LSTAT", "MEDV"], "DOUBLE")
dfts = dfts.cast(["CHAS", "RAD", "TAX"], "INTEGER")
dfts = dfts.to_head("ID")
dfts.head(5).collect()

,ID,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,BLACK,LSTAT,MEDV
0,350,0.02899,40.0,1.25,0,0.429,6.939,34.5,8.7921,1,335,19.7,389.85,5.89,26.6
1,128,0.25915,0.0,21.89,0,0.624,5.693,96.0,1.7883,4,437,21.2,392.11,17.19,16.2
2,351,0.06211,40.0,1.25,0,0.429,6.490,44.4,8.7921,1,335,19.7,396.90,5.98,22.9
3,352,0.07950,60.0,1.69,0,0.411,6.579,35.9,10.7103,4,411,18.3,370.78,5.49,24.1
4,129,0.32543,0.0,21.89,0,0.624,6.431,98.8,1.8125,4,437,21.2,396.90,15.39,18.0


## Create the model
Although we had seen graphically that only a few features had an impact on housing prices, let us use all the features to create a model.  We will then use the model to check for importance of the features.

In [9]:
# We build the model without IDs.  Project only the features and the label.
df = dfts.select(features, label)
model = trees.RandomForestRegressor(connection_context)
model.fit(df, features=features, label=label)

### SQL statements executed
Calling PAL directly would require a number of SQL statements and all that is encapsulated in the Python library functions.

## Model analysis
Let's just see what features are most important.
Note that we are using a sort function.  The property __feature_importances___ is automatically set when the fit() method is called above.

In [10]:
model.feature_importances_.sort(['IMPORTANCE'], desc=True).collect()

,VARIABLE_NAME,IMPORTANCE
0,LSTAT,0.382134
1,RM,0.234494
2,NOX,0.098742
3,CRIM,0.069438
4,TAX,0.059733
5,DIS,0.057167
6,PTRATIO,0.043106
7,INDUS,0.019912
8,AGE,0.013859
9,BLACK,0.010605


__As you can see above, LSTAT, RM, NOX, and PTRATIO seem to have the most impact on prices.__

# Predict using test set
Let us now do some predictions and see how well the model generalizes.

The predict() method always takes a unique identifier to identify the prediction on a specific data row.  This way, the caller (python programmer) can then join with the original data set to get the rest of the values for that unique row.  The test_set has columns of types that PAL does not deal with and therefore the columns are cast to the types that are accepted.

In order to look at the predicted value as well as the true value, the name of the unique identifier for rows in the result table is renamed to PREDICTED_ID.  This result table is joined with the test set so the predicted and true value can be compared.

For the predictions we look at the standard error.  The standard error is defined as the number of standard deviations away the prediction is from the true value.

In [11]:
df_test = test_set.cast(['CRIM', "ZN", "INDUS", "NOX", "RM", "AGE", "DIS", "PTRATIO", "BLACK", "LSTAT", "MEDV"], "DOUBLE")
df_test = df_test.cast(["CHAS", "RAD", "TAX"], "INTEGER")
df_test = df_test.to_head("ID")

In [12]:
# Note that we are renaming the column ID in the result of predict()
result_df = model.predict(df_test, key= 'ID', features=features).rename_columns({'ID': 'PREDICTED_ID'})
# Note the use of join() method to join two tables.
jdf = result_df.join(test_set, '{}."PREDICTED_ID"={}."ID"'.format(result_df.name, test_set.name), how='inner')

### Predictions
Let us look at the predictions.  The predicted values are in 'SCORE' and the actual values are in 'MEDV'.  So, we just rename the 'SCORE' column to 'PREDICTED'

In addition, the column 'CONFIDENCE' is the standard error which is the number of standard deviations away the actual values is from the predicted value.  This column is renamed to 'STANDARD_ERROR'

In [13]:
jdf.select(['ID', 'SCORE', 'MEDV', 'CONFIDENCE']).rename_columns({"CONFIDENCE": "STANDARD_ERROR", "SCORE": "PREDICTED"}).sort("STANDARD_ERROR", desc=False).head(5).collect()

,ID,PREDICTED,MEDV,STANDARD_ERROR
0,20,21.3614,18.2,1.493714
1,22,21.045,19.6,1.519759
2,39,21.5326,24.7,1.521184
3,18,20.9283,17.5,1.569137
4,15,21.6341,18.2,1.615615


### Out of bag error
Let us look at the out of bag errors which is a method of measuring the prediction error.

Here we look at the first 4 rows

In [14]:
model.oob_error_.head(4).collect()

,TREE_INDEX,ERROR
0,0,34.529505
1,1,33.934854
2,2,31.223206
3,3,30.160618


## Scoring
We now score the results from are test data.  The scoring function we use is R^2.

__In the function below, PAL is not invoked but a query is directly executed against data in HANA__

In [15]:
r2_score = model.score(df_test, key='ID', features=features, label=label)
print("r2 score is {}".format(r2_score))

r2 score is 0.6727879210329948


## Model
The model is available and can be saved for later predictions

In [16]:
#  The generated model is in the database.
model.model_.head(4).collect()

,ROW_INDEX,TREE_INDEX,MODEL_CONTENT
0,0,-1,"<PMML version=""4.0"" xmlns=""http://www.dmg.org/..."
1,1,0,"<PMML version=""4.0"" xmlns=""http://www.dmg.org/..."
2,2,1,"<PMML version=""4.0"" xmlns=""http://www.dmg.org/..."
3,3,1,"8"" score=""12.5337"" recordCount=""77"" >\n<Simple..."
